# Understand the model

## General idea

The input data is given and it is expected to get the output data. The input data is video data which is high-demensional time series while the output is new video data generated from learned dynamic texture.

The model consists of two essential stages:
1. Dynamic texture modeling
2. Dynamic texture syntesis

## Dynamic texture modeling

Dynamical texture modeling consists of two steps:
- Dimensionality reduction
- Dynamic texture learning

![title](data/graph.jpg "ShowMyImage")

It can be expressed as:
$$x_{t+1} = f(x_t,A) + n_{x,t}$$
$$y_t = g(x_t,B) + n_{y,t}$$

$y_t$ - column vector unfolded from the frame at time t, $y_t \in R^D$, $D$ - large

$xt$ - latent variable which affects dynamic behaviour, $x_t \in R^Q$, $Q << D$

$g()$ - dimensionality reduction function $R^D -> R^Q$

$f()$ - dynamic modelling function $R^Q -> R^Q$

$n_{x,t}, n_{y,t}$ - represent the noise

$A,B$ - input parameters for functions $f()$ and $g()$

### Dimensionality reduction

Initial dimensionality is N x M where N x M - resolution of an input video file. For instance, it can be 160 x 120 pixels or 19200 values of intensity (in black and white) for every video frame. In such a case curse of dimensionality problem will be faced (data becomes too sparce and distance functions become not accurate). To prevent it dimensionality reduction process must be aplied.

As an reduction algorithm it can be linear aproach (like PCA) or nonlinear. Linear algorithms cannot capture complex dynamic textures. Some nonlinear algorithms produce irreversible mapping and/or different coordinate systems. It is essential to find an algorithm which is free of these weak points. **In this work the reduction function infers by using Gaussian process.**

It is assumed that the dynamic texture sequence $y_i$ is a multivariate Gaussian process indexed by $x_i$ : $P(Y|X,\theta) = f(Y, K_Y, D, N)$, where $Y$ - observed dynamic texture sequence, $X$ - latent variable, $K_Y$ - kernel matrix of latent mapping g(), $D$ - dimensionality, $N$ - number of examples (frames).

To achive nonlinear mapping special covariance function is used (squared exponential):
$$K_Y = k_Y(x_i,x_j) = \theta_1 \exp(-\frac{\theta_2}{2} (x_i - x_j)(x_i - x_j)^T) + \theta_3 \delta_{x_i,x_j}$$

### Dynamic texture learning

The dinamic texture algorithm cannot be linear, because most of dynamic textures are not linear. It can be switching or piecewise linear, but it doesn't work for all dynamic textures. Thus, it is essential to find a more flexible model.

**In this work dynamic texture is modeled using first-order Markov model based on Gaussian process.** The kernel function used in Gaussian process detects the dynamic behavior of the latent variables. The latent dynamic behavior changes among different types of dynamic textures. Thus, a multi-kernel dynamic model can be used: $P(X|\lambda,W) = f(x_1, X, K_X, Q, N, W)$, where $X$ - latent variable, $Q$ - dimensionality of latent variable, $N$ - number of examples (frames), $W$ - kernel functions, $K_X$ - kernel matrix of latent mapping constructed by multi-kernel function:
$$K_X = k_X(x_i,x_j) = 
\sum_{\substack{l = 1}} w_l k_l(x_i,x_j) + w_{\delta} \delta_{x_i,x_j}$$
where $i,j \in [1, N-1]$, $k_l$ - different kernel functions, $l \in [1, M]$, $M$ - number of kernel functions, $w_l$ - weights of kernel functions.

## Dynamic texture syntesis

The goal of this stage is to generate new video data using learned dynamic texture. It can be done by estimating necessary parameters (latent variable vector, observed dynamic texture vector, kernel matrix mapping hyperparameter, weights for kernel functions and different kernel parameters) and then predicting new sequence of dynamic textures.

The generative model is:
$$P(X,Y,\theta,\lambda|W) = P(Y|X,\theta)P(X|\theta,W)P(\theta)P(\lambda)$$

**In this work adopted mean-prediction method based on first-order Markov model using Gaussian prediction is used to syntesise new data.**

# Understand the algorithm 

### MK-GPDM learning

Input - frame Y, 

Output - latent variable X, hyperparameters $\theta$ and $\lambda$, kernel weights $W$

**Step 0:** initialization of X by PCA on Y with Q dimensions, initialization of $\theta$, $\lambda$, $W$

**Step 1:** fix W, optimize $P(Y|X,\theta)P(X|\theta,W)P(\theta)P(\lambda)$ by using SCG and find argmax of $X$, $\theta$, $\lambda$ as the result

**Step 2:** fix $X$, $\theta$, $\lambda$, optimize $\frac{Q}{2} ln|K_X^{-1}|+\frac{1}{2} tr(K_X^{-1} X_{2:N} X_{2:N}^T) + \alpha ||W||_2$ by using gradient descent and find optimum W

**Step 3:** any $w_i \in W$ < 0, $w_i$ <- 0

**Step 4:** normalize W s.t. $\sum_{\substack{i}} = 1$

**Step 5:** repeat steps 1-5 l times, where l - number of frames (kernels, ...) - ?

### SCG

SCG - Scaled conjugate gradient optimization. It uses a scaled conjugate gradients algorithm to find a local minimum of the function F(X) whose gradient is given. Conjugate Gradients algorithm - uses gradients to minimize function $f: R^n -> R$

SCG(F, X, OPTIONS, GRADF, P1, P2, ...)

Here: X - row vector

F - returns a scalar value

The point at which F has a local minimum is returned as X

The function value at that point is returned in OPTIONS(8)

***[params, options, flog] = scg('mk_likelihood', params, options, 'mk_gradient', Y, segments, kern);***

**'mk_likelihood'** - given function F(X)

params - variable needs to be optimized

**'mk_gradient'** - given gradient function GRADF(X)

Y, segments, kern - additional arguments to be passed to F() and GRADF()

## Functions to optimize

### In step 1:

$F(X,\theta,\lambda) = -ln P(X,\theta,\lambda|Y) = \frac{D}{2} ln |K_Y| + \frac{1}{2} tr(K_Y^{-1} Y Y^{-1}) + \frac{Q}{2} ln|K_X| + \frac{1}{2} tr(K_X^{-1} X_{2:N} X_{2:N}^T) + \sum_{\substack{i}}\theta_i + \sum_{\substack{i,j}}(\lambda_i)_j + C$

**Dimensionality analysis:**

$K_Y$ = N x N, while $\frac{D}{2} ln |K_Y|$ = 1 x 1

$Y$ = N x D, $Y^{-1}$ = D x N, thus $K_Y^{-1} Y Y^{-1}$ = N x N, while $\frac{1}{2} tr(K_Y^{-1} Y Y^{-1})$ = 1 x 1

$K_X$ = N-1 x N-1, while $\frac{Q}{2} ln|K_X|$ = 1 x 1

$X$ = N x Q, $X_{2:N}$ = N-1 x Q, thus $K_X^{-1} X_{2:N} X_{2:N}^T$ = N-1 x N-1, while $\frac{1}{2} tr(K_X^{-1} X_{2:N} X_{2:N}^T)$ = 1 x 1

$\sum_{\substack{i}}\theta_i$ = 1 x 1, along with $\sum_{\substack{i,j}}(\lambda_i)_j$ = 1 x 1 and $C$ = 1 x 1

To sum up: $F(X,\theta,\lambda)$ = 1 x 1

### In step 2:

$F(W) = \frac{Q}{2} ln|K_X^{-1}|+\frac{1}{2} tr(K_X^{-1} X_{2:N} X_{2:N}^T) + \alpha ||W||_2$

**Dimensionality analysis:**

$K_X$ = N-1 x N-1, while $\frac{Q}{2} ln|K_X^{-1}|$ = 1 x 1

$X$ = N x Q, $X_{2:N}$ = N-1 x Q, thus $K_X^{-1} X_{2:N} X_{2:N}^T$ = N-1 x N-1, while $\frac{1}{2} tr(K_X^{-1} X_{2:N} X_{2:N}^T)$ = 1 x 1

$W$ = 1 x M, while $\alpha ||W||_2$ = 1 x 1

To sum up: $F(W)$ = 1 x 1

## Kernels

### Latent mapping

$(K_Y)_{i,j} = \theta_1 exp(-\frac{\theta_2}{2}(x_i - x_j)(x_i - x_j)^T) + \theta_3 \delta_{x_i,x_j}$

**Dimensionality analysis:**

$x_i$, $x_j$ = 1 x Q, thus $(x_i - x_j)(x_i - x_j)^T$ = 1 x 1, while $\theta_1 exp(-\frac{\theta_2}{2}(x_i - x_j)(x_i - x_j)^T)$ = 1 x 1 and $\delta_{x_i,x_j}$ = 1 x 1

To sum up: $(K_Y)_{i,j}$ = 1 x 1, thus  $K_Y$ = N x N

### Dynamic modeling

$(K_X)_{i,j} = \sum_{\substack{l = 1..M}} w_l k_l(x_i,x_j) + w_{\delta} \delta_{x_i,x_j}$, where $i,j = 1,..,N-1$

**Dimensionality analysis:**

$k_l(x_i,x_j)$ = 1 x 1, $\delta_{x_i,x_j}$ = 1 x 1

To sum up: $(K_X)_{i,j}$ = 1 x 1, thus  $K_X$ = N-1 x N-1

## Gradients

### Gradient of function 1 with respect to X

$\frac{\partial F}{\partial X_{1:N-1}} = \frac{\partial F}{\partial K_Y} * \frac{\partial K_Y}{\partial X_{1:N-1}} + \frac{\partial F}{\partial K_X} * \frac{\partial K_X}{\partial X_{1:N-1}} + \frac{\partial F}{\partial X_{2:N}} * \frac{\partial X_{2:N}}{\partial X_{1:N-1}}$, where:

$\frac{\partial F}{\partial K_Y} = \frac{D}{2} K_Y^{-T} - \frac{1}{2} (K_Y^{-1} Y Y^{-1} K_Y^{-1})^T$

$\frac{\partial F}{\partial K_X} = \frac{Q}{2} K_X^{-T} - \frac{1}{2} (K_X^{-1} X_{2:N} X_{2:N}^{-1} K_X^{-1})^T$

$\frac{\partial F}{\partial X_{2:N}} = \frac{1}{2} (K_X^{-1} X_{2:N} + K_X^{-T} X_{2:N})$

$\frac{\partial K_Y}{\partial X_{1:N-1}}, \frac{\partial K_X}{\partial X_{1:N-1}}, \frac{\partial X_{2:N}}{\partial X_{1:N-1}}$ - library functions

**Dimensionality analysis:**

$\frac{D}{2} K_Y^{-T}$ = N x N, $\frac{1}{2} (K_Y^{-1} Y Y^{-1} K_Y^{-1})^T$ = N x N, thus $\frac{\partial F}{\partial K_Y}$ = N x N

$\frac{Q}{2} K_X^{-T}$ = N-1 x N-1, $\frac{1}{2} (K_X^{-1} X_{2:N} X_{2:N}^{-1} K_X^{-1})^T$ = N-1 x N-1, thus $\frac{\partial F}{\partial K_X}$ = N-1 x N-1

$\frac{1}{2} (K_X^{-1} X_{2:N} + K_X^{-T} X_{2:N})^T$ = N-1 x Q, thus $\frac{\partial F}{\partial K_Y}$ = N-1 x Q

$\frac{\partial K_Y}{\partial X_{1:N-1}}$ = N-1 x Q, $\frac{\partial K_X}{\partial X_{1:N-1}}$ = N-1 x Q, $\frac{\partial X_{2:N}}{\partial X_{1:N-1}}$ = 1 ?, thus:

$\frac{\partial F}{\partial X_{1:N-1}}$ = N-1 x Q

### Gradient of function 1 with respect to $\theta$

$\frac{\partial F}{\partial \theta} = \frac{D}{2} tr(K_Y^{-1} \frac{\partial K_Y}{\partial \theta}) - \frac{1}{2} (K_Y^{-1} Y Y^T K_Y^{-1})^T \frac{\partial K_Y}{\partial \theta} + \frac{1}{\theta}$, where
$\frac{\partial K_Y}{\partial \theta}$ - library function

**Dimensionality analysis:**

$\frac{D}{2} tr(K_Y^{-1} \frac{\partial K_Y}{\partial \theta})$ = 1 x 1

$(K_Y^{-1} Y Y^T K_Y^{-1})^T$ = N x N

$\frac{\partial K_Y}{\partial \theta}$ - ?

$\theta$ = 1 x 3 - ?

$\frac{\partial F}{\partial \theta}$ - ?

### Gradient of function 1 with respect to $\lambda$

$\frac{\partial F}{\partial \lambda} = \frac{Q}{2} tr(K_X^{-1} \frac{\partial K_X}{\partial \lambda}) - \frac{1}{2} (K_X^{-1} X_{2:N} X_{2:N}^T K_X^{-1})^T \frac{\partial K_X}{\partial \lambda} + \frac{1}{\lambda}$, where
$\frac{\partial K_X}{\partial \lambda}$ - library function

**Dimensionality analysis:**

$\frac{Q}{2} tr(K_X^{-1} \frac{\partial K_X}{\partial \lambda})$ = 1 x 1

$(K_X^{-1} X_{2:N} X_{2:N}^T K_X^{-1})^T$ = N-1 x N-1

$\frac{\partial K_X}{\partial \lambda}$ - ?

$\lambda$ = $M$ x $M_l$

$\frac{\partial F}{\partial \lambda}$ - ?

# Understand the code

### DEMO_FOR_MKGPDM.M

***% Load DT sequences as training samples***

mov - original video


Y - vector of frames from original video after the preprocessing, 250x10800

k - number of original frames (=250 = 10 seconds)


***%% Set the paramters for SCG***

opt - vector of parameters (=20 iterations)

N - number of frames (=250)

D - dimensionality (=120x90)

Q - dimensionality of latent space (=20)


***%% Initialize the latent variable using PCA***

X - vector of frames in latent space, 250x20

kern - structure with 6 kernel functions and their parameters inside


***%% Initialize the hyperparameters***

hyperpara_Kx, hyperpara_Ky - hyperparameters Kx and Ky (instead of theta and lambda - ?)

weight - vector W with weights for kernels


***%% Optimization***

<font color='red'>**mk_gpdm**</font> - optimization function for Step 1 and Step 2


***%% Prediction***

<font color='red'>**mk_prediction**</font> - optimization function for Step 2

predFrames - number of frames to synthesis (=500 = 20 seconds)

Ysample - output result in complex form

Ymean - mean of the results (?)


***%% Output Results***

Ysample1 - real values of Ysample

I - normalized frame

synth_Result - result matrix 500x10800

?_synth_Result.mat - output file

### MK_GPDM.M

N, D, Q - the same

lnHyperpara_Ky, lnHyperpara_Kx - theta, theta_p which correspond to alpha, beta

iters - number of iterations

<font color='red'>**scg**</font> - optimization function for optimizing hyperparameters and X, weights

**mk_paramsDecompose** - parameters decomposition

**mk_kernExpandParam** - kernels decomposition

**mk_kernWeightExtract** - returns weights in column vector form

**mk_weightsConstrain** - Step 3 of the algorithm (for any $w_i$ ∈ W < 0: $w_i$ <- 0)

**mk_updateKernWeight** - assigns kernel weights according the input

### MK_PREDICTION.M

**computeKernel** - сomputes the kernel matrix for data X with parameters theta (Ky and invKy)

**mk_priorIO** - first order Markov model (calculation of Xin)

mk_kernExpandParam, mk_updateKernWeight - the same as before

**mk_computeCompoundKernel** - calculates the results of multi-core functions (calculation of Kx)

pdinv - computes inverse of positive definite matrix

simSteps - number of frames generated

**mk_simulatedynamics** - generates frames of hidden variables

**mk_sampleReconstruction** - reconstructs hidden variables to a high-dimensional space